In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.utils.test_scripts import *
from pkg.planning.pddlstream.plan_rnb import *

In [3]:
rtype = "indy7gripper"
dat_root = "stowing-deep"
res_root = "stowing-deep-result"
dat_dir = "20210829-050034"
file_option = "obj_3_hard"
data_idx = 0
cname = "Full"

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100
MAX_SKELETONS = 10

GRASP_SAMPLE = 100
STABLE_SAMPLE = 100
SEARCH_SAMPLE_RATIO = 10

VISUALIZE = True
SHOW_STATE = False
PLAY_RESULT = True
USE_PYBULLET_GUI = True



CLEARANCE = 1e-3
TOOL_NAME="grip0"
ROBOT_TYPE = {e.name: e for e in RobotType}[rtype]

In [4]:
DATASET_PATH = create_data_dirs(dat_root, rtype, dat_dir)
RESULTSET_PATH = create_data_dirs(res_root, rtype, dat_dir)
print("-"*50)
print("DATASET_PATH: {}".format(DATASET_PATH))
print("RESULTSET_PATH: {}".format(RESULTSET_PATH))
print("-"*50)

--------------------------------------------------
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/stowing-deep/indy7gripper/20210829-050034
RESULTSET_PATH: /home/rnb/Projects/rnb-planning/data/stowing-deep-result/indy7gripper/20210829-050034
--------------------------------------------------


In [5]:
ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_builder, pscene = prepare_single_robot_scene(ROBOT_TYPE, ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, VISUALIZE=VISUALIZE)
crob, gscene = pscene.combined_robot, pscene.gscene
crob.home_pose = HOME_POSE
crob.home_dict = list2dict(HOME_POSE, gscene.joint_names)

fname = "data_%s_%02d.pkl" % (file_option, data_idx)
print(fname)
file_gtems = os.path.join(DATASET_PATH, fname)
initial_state = load_saved_scene(pscene, file_gtems, VISUALIZE=VISUALIZE)

gscene.NAME_DICT['obj_0'].color = (1,0,0,1)
gscene.update_markers_all()

mplan = MoveitPlanner(pscene)
checkers = get_checkers_by_case_name(cname, pscene)

mplan.motion_filters = checkers

connection command:
indy0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
data_obj_3_hard_00.pkl


In [6]:
gscene.NAME_DICT["pole"].collision = False

In [7]:
obj_pscene = pscene.subject_dict[pscene.subject_name_list[0]]
obj_pscene.geometry.color = (0.8, 0.2, 0.2, 1)
goal_pairs=[(obj_pscene.oname, 'gp')]

gtimer = GlobalTimer.instance()
gtimer.reset()

res, plan, log_dict = solve_in_pddlstream(pscene, mplan, ROBOT_NAME, TOOL_NAME, HOME_POSE, goal_pairs, 
                        TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS, 
                        GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO, 
                        use_pybullet_gui=USE_PYBULLET_GUI)

# save_pickle(os.path.join(RESULTSET_PATH, "result_%s_%02d_%s.pkl" % (file_option, data_idx, cname)), log_dict)

print("------- Result {} ({}): {} s -------".format(fname, cname, log_dict["plan_time"]))
print("==========================================================")
print("==========================================================")
print(gtimer)
print("==========================================================")
print("==========================================================")


[Pybullet] Load urdf from /home/rnb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
Objects: {1L: 'obj_0', 2L: 'gp', 3L: 'obj_2', 4L: 'obj_1', 5L: 'floor', 6L: 'base', 7L: 'wp', 8L: 'obs_0'}
================ MAKE PROBLEM ======================
IK checkers: ['GraspChecker', 'ReachChecker']
MP checkers: ['GraspChecker', 'ReachChecker', 'LatticedChecker']
timeout motion : 5
Robot: 0
Movable: [3L, 4L, 1L]
Fixed: [2, 5, 6, 7, 8]
body 3 - surface 2
body 3 - surface 5
body 3 - surface 6
body 3 - surface 7
body 3 - surface 8
body 4 - surface 2
body 4 - surface 5
body 4 - surface 6
body 4 - surface 7
body 4 - surface 8
body 1 - surface 2
body 1 - surface 5
body 1 - surface 6
body 1 - surface 7
body 1 - surface 8
Init: [('CanMove',), ('Conf', q0), ('AtConf', q0), ('HandEmpty',), ('Graspable', 3L), ('Pose', 3L, p0), ('AtPose', 3L, p0), ('Stackable', 3L, 2), ('Stackable', 3L, 5), ('Stackable', 3L, 6), ('Stackable', 3L, 7), ('Stackable', 3L, 8), ('Graspable', 4L), ('Pose', 4L, p1), ('A

KeyboardInterrupt: 

In [ ]:
SolutionStore.last_log['run_time']

In [7]:
if VISUALIZE and PLAY_RESULT and res:
    play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state,
                   body_names=log_dict['body_names'], plan=plan, SHOW_PERIOD=0.01)


## Old version

In [7]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [8]:
gscene.NAME_DICT["pole"].collision = True

In [9]:
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

In [11]:
from_state = initial_state.copy(pscene)
from_state.Q = HOME_POSE
goal_nodes = [('gp', 'wp', 'wp')]
ppline.search(from_state, goal_nodes, verbose=True, display=False, 
              timeout_loop=100, multiprocess=False, timeout=1, add_homing=False)

try: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
Motion Filer Failure: GraspChecker
result: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0')
Motion Filer Failure: GraspChecker
result: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
try: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp')
Motion Filer Failure: GraspChecker
result: 0 - ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
try: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0')
Motion Filer Failure: GraspChecker
result: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
try: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0')
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
try: 0 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp')
Motion Filer Failure: GraspChecker
result: 0 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
try: 0 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0')
try transition motion
transit

transition motion tried: True
result: 4 - ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = success
branching: 4->6 (6.03/100.0 s, steps/err: 108(646.018981934 ms)/0.00179017849872)
try: 0 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp')
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
try: 6 - ('wp', 'grip0', 'gp')->('wp', 'gp', 'gp')
try transition motion
transition motion tried: True
result: 6 - ('wp', 'grip0', 'gp')->('wp', 'gp', 'gp') = success
branching: 6->7 (7.29/100.0 s, steps/err: 87(190.438985825 ms)/0.00115627240796)
try: 6 - ('wp', 'grip0', 'gp')->('wp', 'wp', 'gp')
Motion Filer Failure: ReachChecker
result: 6 - ('wp', 'grip0', 'gp')->('wp', 'wp', 'gp') = fail
try: 4 - ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp')
Motion Filer Failure: GraspChecker
result: 4 - ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
try: 4 - ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp')
Motion Filer Failure: GraspChecker
result: 4 -

result: 1 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = success
branching: 1->12 (14.48/100.0 s, steps/err: 60(186.719179153 ms)/0.0020919645629)
try: 12 - ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0')
try transition motion
transition motion tried: True
result: 12 - ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0') = success
branching: 12->13 (14.79/100.0 s, steps/err: 97(311.277151108 ms)/0.00150911704799)
try: 11 - ('wp', 'grip0', 'gp')->('wp', 'gp', 'gp')
Motion Filer Failure: ReachChecker
result: 11 - ('wp', 'grip0', 'gp')->('wp', 'gp', 'gp') = fail
try: 13 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp')
Motion Filer Failure: GraspChecker
result: 13 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
try: 9 - ('wp', 'gp', 'gp')->('grip0', 'gp', 'gp')
Motion Filer Failure: GraspChecker
result: 9 - ('wp', 'gp', 'gp')->('grip0', 'gp', 'gp') = fail
try: 9 - ('wp', 'gp', 'gp')->('grip0', 'gp', 'gp')
try transition motion
transition motion tried: False
Motion Plan Failure
result: 9 - ('wp', 'gp', 'gp')->

result: 5 - ('grip0', 'wp', 'gp')->('wp', 'wp', 'gp') = success
branching: 5->19 (22.71/100.0 s, steps/err: 10(354.180812836 ms)/0.00167737014377)
try: 19 - ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp')
try transition motion
transition motion tried: False
Motion Plan Failure
result: 19 - ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
try: 14 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0')
Motion Filer Failure: ReachChecker
result: 14 - ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
try: 15 - ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0')
try transition motion
transition motion tried: True
result: 15 - ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0') = success
branching: 15->20 (24.02/100.0 s, steps/err: 50(251.81889534 ms)/0.00140898245342)
try: 20 - ('gp', 'wp', 'grip0')->('gp', 'wp', 'wp')
Motion Filer Failure: GraspChecker
result: 20 - ('gp', 'wp', 'grip0')->('gp', 'wp', 'wp') = fail
try: 1 - ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp')
Motion Filer Failure: ReachChecker
result: 1 - ('wp', 'wp', 'grip

In [12]:
snode_schedule = ppline.tplan.get_best_schedule()

In [13]:
ppline.play_schedule(snode_schedule)

('wp', 'wp', 'wp')->('wp', 'wp', 'wp')
('wp', 'wp', 'wp')->('wp', 'wp', 'grip0')
('wp', 'wp', 'grip0')->('wp', 'wp', 'gp')
('wp', 'wp', 'gp')->('grip0', 'wp', 'gp')
('grip0', 'wp', 'gp')->('gp', 'wp', 'gp')
('gp', 'wp', 'gp')->('gp', 'wp', 'grip0')
('gp', 'wp', 'grip0')->('gp', 'wp', 'wp')


In [10]:
pscene.set_object_state(initial_state)

In [15]:
if VISUALIZE and PLAY_RESULT and res:
    play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state,
                   body_names=body_names, plan=plan, SHOW_PERIOD=0.01)

s_builder.xcustom.clear()